In [ ]:
!pip install neo4j pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=436430c40cc5311c3d0437ca1875f732a202dcf7543526a978330604a8af899f
  Stored in directory: /root/.cache/pip/wheels/30/db/9b/2cfde1fa33145219c0322f299b604daf5aba2ed443a7ed5f07
Successfully built neo4j


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/combined_data.csv')

# Display the first few rows of the DataFrame to understand its structure
print(df.head())


   rating                                              title parent_asin  \
0     5.0  Tyger Auto T1 Soft Roll Up Truck Bed Tonneau C...  B0BV88374L   
1     5.0  Tailgate Insert Letters Compatible with F150 R...  B0B2WGS5ND   
2     1.0  uxcell Adhesive 3" Round Side Rearview Blind S...  B00A0GV20Q   
3     5.0  Crevelle Custom Fits 2009-2020 Nissan 370Z / 3...  B08C27WWVG   
4     5.0  Pre-Painted Trunk Spoiler Compatible with 2009...  B0719J5ZNY   

                        user_id  average_rating  rating_number  price  \
0  AHMUTK5MAIDB7NQZQYH5NIOPT6BQ             4.8         5239.0  207.0   
1  AFE337D2J37YRU5U6MVTVKNDKWDA             5.0            1.0    NaN   
2  AGMVX6M2O2J7L63ATNZA2KBWHIAQ             3.7            8.0    NaN   
3  AHQOKY3AQJG5WK4QGPCIVRA2UEBQ             4.4           65.0  95.95   
4  AEZBUG7X75XAZ6MSPQ43QWY2WBBQ             4.6           28.0    NaN   

              store                                         categories  
0        Tyger Auto  Automotive

<ipython-input-5-4a5030885a7e>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/combined_data.csv')


In [ ]:
from neo4j import GraphDatabase

database_name = "neo4j"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687/apan5400" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")


Successfully connected to Neo4j!


In [ ]:
# Display the column names in the DataFrame
print(df.columns)


Index(['rating', 'title', 'parent_asin', 'user_id', 'average_rating',
       'rating_number', 'price', 'store', 'categories'],
      dtype='object')


In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

df = pd.read_csv('/combined_data.csv')


def create_graph(tx, title, average_rating, price, user_id, categories, store, rating):
    # Replace NaN values in categories with a default string, such as 'Unknown Category'
    categories = 'Unknown Category' if pd.isna(categories) else categories

    # Replace NaN values in store with a default string, such as 'Unknown Store'
    store = 'Unknown Store' if pd.isna(store) else store

    # Replace NaN values in title with a default string, such as 'Unknown Title'
    title = 'Unknown Title' if pd.isna(title) else title

    # Replace NaN values in price with a default value, such as 0
    price = 0 if pd.isna(price) else price

    # Replace NaN values in average_rating with a default value, such as 0
    average_rating = 0 if pd.isna(average_rating) else average_rating

    query = """
    MERGE (p:Product {title: $title, average_rating: COALESCE($average_rating, 0), price: $price})
    MERGE (u:User {user_id: $user_id})
    MERGE (c:Category {name: $categories})
    MERGE (s:Store {name: $store})
    MERGE (u)-[:PURCHASED {rating: $rating}]->(p)
    MERGE (p)-[:BELONGS_TO]->(c)
    MERGE (p)-[:SOLD_BY]->(s)
    """
    tx.run(query, title=title, average_rating=average_rating, price=price,
           user_id=user_id, categories=categories, store=store, rating=rating)

database_name = "neo4j"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687"

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
    for index, row in df.iterrows():
        session.write_transaction(create_graph,
                                  row.get('title'),
                                  row.get('average_rating'),
                                  row.get('price'),
                                  row.get('user_id'),
                                  row.get('categories'),
                                  row.get('store'),
                                  row.get('rating'))

print("Data insertion complete.")

<ipython-input-14-02fdb47f0130>:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/combined_data.csv')
<ipython-input-14-02fdb47f0130>:45: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph,
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address)